In [1]:
from lsst.sims.ocs.observatory import MainObservatory
from lsst.sims.ocs.configuration import ObservingSite, Observatory
from lsst.ts.observatory.model import ObservatoryModel
import numpy as np

In [2]:
class target_spoof(object):
    """Make a dummy class that can pass as a  SALPY_scheduler.targetC
    """
    def __init__(self, targetId=None, fieldId=None, filtername=None, ra=0., dec=0.,
                angle=0., num_exposures=2, exposure_times=15.):
        self.targetId = targetId
        self.fieldId = fieldId
        self.filter = filtername
        self.ra = ra
        self.dec = dec
        self.angle = angle
        self.num_exposures = num_exposures
        self.exposure_times = [exposure_times]*num_exposures

In [3]:
class observation_spoof(object):
    def __init__(self, observationId=None, start_time=None, start_mjd=None, start_lst=None,
                 targetId=None, num_proposals=None, proposal_Ids=None, fieldId=None,
                groupId=None, filter='r', ra=0., dec=0., angle=0., num_exposures=2):
        self.observationId = observationId
        self.start_time = start_time
        self.start_mjd = start_mjd
        self.start_lst = np.degrees(start_lst)
        self.targetId = targetId
        self.num_proposals = num_proposals
        if proposal_Ids is None:
            self.proposal_Ids = []
        else:
            self.proposal_Ids = proposal_Ids
        self.fieldId = fieldId
        self.groupId = groupId
        self.filter = filter
        self.ra = ra
        self.dec = dec
        self.angle = angle
        self.num_exposures = num_exposures
        

In [4]:
class time_handler_spoof(object):
    def __init__(self, start_time=0):
        self.current_timestamp = 0.
    def update_time(self, intime):
        self.current_timestamp += intime


In [5]:
targ = target_spoof(dec=-15.)

In [6]:
site = ObservingSite()

In [22]:
mo = MainObservatory(site)
mo.configure(Observatory())

In [43]:
mo.start_night(10, 3652)
# Make sure I can set the rotator angle
mo.model.params.rotator_followsky = True

In [44]:
target = target_spoof(ra=0., dec=-30., angle=45.)
# Slew to my target
print mo.slew(target)
# Check that the current state is what I wanted
print mo.model.current_state.ra, mo.model.current_state.dec, mo.model.current_state.ang

(18.691900924272062, 'seconds')
0.0 -30.0 45.0


In [45]:
# Slew to a new field 20 degrees away
target = target_spoof(ra=0., dec=-20., angle=45.)
# Slew to my target
print mo.slew(target)
print mo.model.current_state.ra, mo.model.current_state.dec, mo.model.current_state.ang

(16.713047678761768, 'seconds')
0.0 -20.0 45.0


In [46]:
# Let's try slewing to the same spot
print mo.slew(target)
print mo.model.current_state.ra, mo.model.current_state.dec, mo.model.current_state.ang

(2.0, 'seconds')
0.0 -20.0 45.0


In [47]:
mo.model.params.rotator_followsky

True

In [48]:
# Can I give a slew command and just change the angle?
target = target_spoof(ra=0., dec=-20., angle=90.)
print mo.slew(target)
print mo.model.current_state.ra, mo.model.current_state.dec, mo.model.current_state.ang

(16.35714285714286, 'seconds')
0.0 -20.0 90.0


In [49]:
mo.model.current_state.tracking

True

In [16]:
mo.model.park()
print mo.model.current_state.ang_rad
print mo.slew(target)
print mo.model.current_state.ang_rad
print mo.slew(target)
print mo.model.current_state.ang_rad

3.14159265359
(56.542464411068295, 'seconds')
2.41434415292
(2.0, 'seconds')
2.41434415292


In [17]:
mo.model.current_state.rot_rad

6.279649335666874

In [18]:
model = ObservatoryModel(site)

In [19]:
mo.model.current_state.telrot

-0.2025962441569066

In [50]:
mo.model.params.readouttime

2.0

In [51]:
mo.model.params.readouttime=0.
target = target_spoof(ra=0., dec=-20., angle=90.)
print mo.slew(target)
print mo.model.current_state.ra, mo.model.current_state.dec, mo.model.current_state.ang
print mo.slew(target)

(3.190262418684142e-07, 'seconds')
0.0 -20.0 90.0
(1.7052673229045467e-07, 'seconds')
